In [10]:
# Import necessary libraries
import splitfolders
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report
import numpy as np
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

# Split dataset into train (70%), val (20%), test (10%) with stratified sampling
original_dataset_dir = "./dataset"
output_dir = "./datasplit"

splitfolders.ratio(
    original_dataset_dir,
    output=output_dir,
    seed=42,
    ratio=(0.7, 0.2, 0.1),
    move=False,
)

# Define paths
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "val")
test_dir = os.path.join(output_dir, "test")

# Image dimensions and parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.8, 1.2),
    horizontal_flip=True,
    fill_mode="nearest",
)

# Rescale for validation and testing
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
)


Found 483 images belonging to 4 classes.
Found 137 images belonging to 4 classes.
Found 74 images belonging to 4 classes.


In [11]:


# Verify class distribution
class_counts = np.bincount(train_generator.classes)
print(f"Class counts in training set: {class_counts}")

# Load ResNet50 as base model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze the base model

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)  # Custom layer with reduced units
x = Dropout(0.3)(x)  # Regularization
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

# Build the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

Class counts in training set: [117 125 116 125]


In [12]:


# Callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6)

# Train the model with frozen base layers
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
)


Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 71s 4s/step - accuracy: 0.1997 - loss: 1.8445 - val_accuracy: 0.2482 - val_loss: 1.3998 - learning_rate: 1.0000e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.2446 - loss: 1.5631 - val_accuracy: 0.2117 - val_loss: 1.3843 - learning_rate: 1.0000e-04
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.2089 - loss: 1.5553 - val_accuracy: 0.2336 - val_loss: 1.3818 - learning_rate: 1.0000e-04
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.2290 - loss: 1.5231 - val_accuracy: 0.3139 - val_loss: 1.3740 - learning_rate: 1.0000e-04
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.2388 - loss: 1.5165 - val_accuracy: 0.2847 - val_loss: 1.3743 - learning_rate: 1.0000e-04
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.2535 - loss: 1.4567 - val_accuracy: 0.3066 - val_loss: 1.3717 - learning_rate: 1.0000e-04
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 46s 3s/step - accuracy: 0.3306 - loss:

In [13]:

# Fine-tuning the model
base_model.trainable = True  # Unfreeze the base model

# Optionally freeze the first N layers
for layer in base_model.layers[:100]:
    layer.trainable = False

# Re-compile with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

# Fine-tune the model
history_fine_tune = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
)


Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 116s 6s/step - accuracy: 0.2575 - loss: 1.8626 - val_accuracy: 0.3285 - val_loss: 1.3649 - learning_rate: 1.0000e-05
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 85s 5s/step - accuracy: 0.2654 - loss: 1.4319 - val_accuracy: 0.2993 - val_loss: 1.4049 - learning_rate: 1.0000e-05
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 85s 5s/step - accuracy: 0.2705 - loss: 1.4336 - val_accuracy: 0.2920 - val_loss: 1.4339 - learning_rate: 1.0000e-05
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 94s 6s/step - accuracy: 0.2679 - loss: 1.4307 - val_accuracy: 0.2701 - val_loss: 1.4587 - learning_rate: 1.0000e-05
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 88s 5s/step - accuracy: 0.3583 - loss: 1.3636 - val_accuracy: 0.2628 - val_loss: 1.4787 - learning_rate: 5.0000e-06
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 89s 6s/step - accuracy: 0.3748 - loss: 1.3709 - val_accuracy: 0.2555 - val_loss: 1.4740 - learning_rate: 5.0000e-06
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 86s 6s/step - accuracy: 0.3445 - loss

In [14]:

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

# Generate Classification Report
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_classes, target_names=list(test_generator.class_indices.keys())))

3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.2310 - loss: 1.3833
Test Accuracy: 0.24
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step
              precision    recall  f1-score   support

       Angry       1.00      0.06      0.11        18
        Fear       0.21      0.42      0.28        19
       Happy       0.00      0.00      0.00        18
         Sad       0.26      0.47      0.33        19

    accuracy                           0.24        74
   macro avg       0.37      0.24      0.18        74
weighted avg       0.36      0.24      0.18        74



c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,